In [ ]:
# https://github.com/huggingface/transformers/tree/master/examples/tensorflow/text-classification
# Das in die requirements.txt-Datei
# datasets >= 1.1.3
# sentencepiece != 0.1.92
# protobuf
# tensorflow >= 2.3

In [ ]:
!pip install -r /content/requirements.txt
!pip install transformers
!pip install tensorflow-addons

     |████████████████████████████████| 306 kB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 47.4 MB/s 
     |████████████████████████████████| 61 kB 486 kB/s 
     |████████████████████████████████| 243 kB 50.9 MB/s 
     |████████████████████████████████| 1.1 MB 40.3 MB/s 
     |████████████████████████████████| 132 kB 56.1 MB/s 
     |████████████████████████████████| 160 kB 53.8 MB/s 
     |████████████████████████████████| 271 kB 53.0 MB/s 
     |████████████████████████████████| 192 kB 56.9 MB/s 
     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
     |████████████████████████████████| 895 kB 53.0 MB/s 
     |████████████████████████████████| 3.3 MB 33.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [ ]:
%%writefile text_class.py
#!/usr/bin/env python
# coding=utf-8
# Copyright 2021 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning the library models for sequence classification."""
# You can also adapt this script on your own text classification task. Pointers for this are left as comments.

import logging
import os
import sys
from dataclasses import dataclass, field
from pathlib import Path
from typing import Optional
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import numpy as np
from datasets import load_dataset
from keras import backend as K
from sklearn.metrics import classification_report

from transformers import (
    AutoConfig,
    AutoTokenizer,
    HfArgumentParser,
    PretrainedConfig,
    TFAutoModelForSequenceClassification,
    TFTrainingArguments,
    set_seed,
)
from transformers.file_utils import CONFIG_NAME, TF2_WEIGHTS_NAME


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"  # Reduce the amount of console output from TF
import tensorflow as tf  # noqa: E402
import tensorflow_addons as tfa


logger = logging.getLogger(__name__)


# region Helper classes
class SavePretrainedCallback(tf.keras.callbacks.Callback):
    # Hugging Face models have a save_pretrained() method that saves both the weights and the necessary
    # metadata to allow them to be loaded as a pretrained model in future. This is a simple Keras callback
    # that saves the model with this method after each epoch.
    def __init__(self, output_dir, **kwargs):
        super().__init__()
        self.output_dir = output_dir

    def on_epoch_end(self, epoch, logs=None):
        self.model.save_pretrained(self.output_dir)


def convert_dataset_for_tensorflow(
    dataset, non_label_column_names, batch_size, dataset_mode="variable_batch", shuffle=True, drop_remainder=True
):
    """Converts a Hugging Face dataset to a Tensorflow Dataset. The dataset_mode controls whether we pad all batches
    to the maximum sequence length, or whether we only pad to the maximum length within that batch. The former
    is most useful when training on TPU, as a new graph compilation is required for each sequence length.
    """

    def densify_ragged_batch(features, label=None):
        features = {
            feature: ragged_tensor.to_tensor(shape=batch_shape[feature]) for feature, ragged_tensor in features.items()
        }
        if label is None:
            return features
        else:
            return features, label

    feature_keys = list(set(dataset.features.keys()) - set(non_label_column_names + ["label"]))
    if dataset_mode == "variable_batch":
        batch_shape = {key: None for key in feature_keys}
        data = {key: tf.ragged.constant(dataset[key]) for key in feature_keys}
    elif dataset_mode == "constant_batch":
        data = {key: tf.ragged.constant(dataset[key]) for key in feature_keys}
        batch_shape = {
            key: tf.concat(([batch_size], ragged_tensor.bounding_shape()[1:]), axis=0)
            for key, ragged_tensor in data.items()
        }
    else:
        raise ValueError("Unknown dataset mode!")

    if "label" in dataset.features:
        labels = tf.convert_to_tensor(np.array(dataset["label"]))
        tf_dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    else:
        tf_dataset = tf.data.Dataset.from_tensor_slices(data)
    if shuffle:
        tf_dataset = tf_dataset.shuffle(buffer_size=len(dataset))
    tf_dataset = tf_dataset.batch(batch_size=batch_size, drop_remainder=drop_remainder).map(densify_ragged_batch)
    return tf_dataset


# endregion


# region Command-line arguments
@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    train_file: Optional[str] = field(
        default=None, metadata={"help": "A csv or a json file containing the training data."}
    )
    validation_file: Optional[str] = field(
        default=None, metadata={"help": "A csv or a json file containing the validation data."}
    )
    test_file: Optional[str] = field(default=None, metadata={"help": "A csv or a json file containing the test data."})

    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached preprocessed datasets or not."}
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to `max_seq_length`. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch."
            "Data will always be padded when using TPUs."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_val_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of validation examples to this "
            "value if set."
        },
    )
    max_test_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of test examples to this "
            "value if set."
        },
    )

    def __post_init__(self):
        train_extension = self.train_file.split(".")[-1].lower() if self.train_file is not None else None
        validation_extension = (
            self.validation_file.split(".")[-1].lower() if self.validation_file is not None else None
        )
        test_extension = self.test_file.split(".")[-1].lower() if self.test_file is not None else None
        extensions = {train_extension, validation_extension, test_extension}
        extensions.discard(None)
        assert len(extensions) != 0, "Need to supply at least one of --train_file, --validation_file or --test_file!"
        assert len(extensions) == 1, "All input files should have the same file extension, either csv or json!"
        assert "csv" in extensions or "json" in extensions, "Input files should have either .csv or .json extensions!"
        self.input_file_extension = extensions.pop()


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )


# endregion


def main():
    # region Argument parsing
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TFTrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    output_dir = Path(training_args.output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    # endregion

    # region Checkpoints
    # Detecting last checkpoint.
    checkpoint = None
    if len(os.listdir(training_args.output_dir)) > 0 and not training_args.overwrite_output_dir:
        if (output_dir / CONFIG_NAME).is_file() and (output_dir / TF2_WEIGHTS_NAME).is_file():
            checkpoint = output_dir
            logger.info(
                f"Checkpoint detected, resuming training from checkpoint in {training_args.output_dir}. To avoid this"
                " behavior, change the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
            )
        else:
            raise ValueError(
                f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                "Use --overwrite_output_dir to continue regardless."
            )

    # endregion

    # region Logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        handlers=[logging.StreamHandler(sys.stdout)],
    )
    logger.setLevel(logging.INFO)

    logger.info(f"Training/evaluation parameters {training_args}")
    # endregion

    # region Loading data
    # For CSV/JSON files, this script will use the 'label' field as the label and the 'sentence1' and optionally
    # 'sentence2' fields as inputs if they exist. If not, the first two fields not named label are used if at least two
    # columns are provided. Note that the term 'sentence' can be slightly misleading, as they often contain more than
    # a single grammatical sentence, when the task requires it.
    #
    # If the CSVs/JSONs contain only one non-label column, the script does single sentence classification on this
    # single column. You can easily tweak this behavior (see below)
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    data_files = {"train": data_args.train_file, "validation": data_args.validation_file, "test": data_args.test_file}
    data_files = {key: file for key, file in data_files.items() if file is not None}

    for key in data_files.keys():
        logger.info(f"Loading a local file for {key}: {data_files[key]}")

    if data_args.input_file_extension == "csv":
        # Loading a dataset from local csv files
        datasets = load_dataset("csv", data_files=data_files, cache_dir=model_args.cache_dir)
    else:
        # Loading a dataset from local json files
        datasets = load_dataset("json", data_files=data_files, cache_dir=model_args.cache_dir)
    # See more about loading any type of standard or custom dataset at
    # https://huggingface.co/docs/datasets/loading_datasets.html.
    # endregion

    # region Label preprocessing
    # If you've passed us a training set, we try to infer your labels from it
    if "train" in datasets:
        # By default we assume that if your label column looks like a float then you're doing regression,
        # and if not then you're doing classification. This is something you may want to change!
        is_regression = datasets["train"].features["label"].dtype in ["float32", "float64"]
        if is_regression:
            num_labels = 1
        else:
            # A useful fast method:
            # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.unique
            label_list = datasets["train"].unique("label")
            label_list.sort()  # Let's sort it for determinism
            num_labels = len(label_list)
    # If you haven't passed a training set, we read label info from the saved model (this happens later)
    else:
        num_labels = None
        label_list = None
        is_regression = None
    # endregion

    # region Load model config and tokenizer
    if checkpoint is not None:
        config_path = training_args.output_dir
    elif model_args.config_name:
        config_path = model_args.config_name
    else:
        config_path = model_args.model_name_or_path
    if num_labels is not None:
        config = AutoConfig.from_pretrained(
            config_path,
            num_labels=num_labels,
            cache_dir=model_args.cache_dir,
            revision=model_args.model_revision,
            use_auth_token=True if model_args.use_auth_token else None,
        )
    else:
        config = AutoConfig.from_pretrained(
            config_path,
            cache_dir=model_args.cache_dir,
            revision=model_args.model_revision,
            use_auth_token=True if model_args.use_auth_token else None,
        )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    # endregion

    # region Dataset preprocessing
    # Again, we try to have some nice defaults but don't hesitate to tweak to your use case.
    column_names = {col for cols in datasets.column_names.values() for col in cols}
    non_label_column_names = [name for name in column_names if name != "label"]
    if "sentence1" in non_label_column_names and "sentence2" in non_label_column_names:
        sentence1_key, sentence2_key = "sentence1", "sentence2"
    elif "sentence1" in non_label_column_names:
        sentence1_key, sentence2_key = "sentence1", None
    else:
        if len(non_label_column_names) >= 2:
            sentence1_key, sentence2_key = non_label_column_names[:2]
        else:
            sentence1_key, sentence2_key = non_label_column_names[0], None

    if data_args.max_seq_length > tokenizer.model_max_length:
        logger.warning(
            f"The max_seq_length passed ({data_args.max_seq_length}) is larger than the maximum length for the"
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )
    max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

    # Ensure that our labels match the model's, if it has some pre-specified
    if "train" in datasets:
        if not is_regression and config.label2id != PretrainedConfig(num_labels=num_labels).label2id:
            label_name_to_id = config.label2id
            if list(sorted(label_name_to_id.keys())) == list(sorted(label_list)):
                label_to_id = label_name_to_id  # Use the model's labels
            else:
                logger.warning(
                    "Your model seems to have been trained with labels, but they don't match the dataset: ",
                    f"model labels: {list(sorted(label_name_to_id.keys()))}, dataset labels: {list(sorted(label_list))}."
                    "\nIgnoring the model labels as a result.",
                )
                label_to_id = {v: i for i, v in enumerate(label_list)}
        elif not is_regression:
            label_to_id = {v: i for i, v in enumerate(label_list)}
        else:
            label_to_id = None
        # Now we've established our label2id, let's overwrite the model config with it.
        config.label2id = label_to_id
        if config.label2id is not None:
            config.id2label = {id: label for label, id in label_to_id.items()}
        else:
            config.id2label = None
    else:
        label_to_id = config.label2id  # Just load the data from the model

    if "validation" in datasets and config.label2id is not None:
        validation_label_list = datasets["validation"].unique("label")
        for val_label in validation_label_list:
            assert val_label in label_to_id, f"Label {val_label} is in the validation set but not the training set!"

    def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

    def preprocess_function(examples):
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, max_length=max_seq_length, truncation=True)

        # Map labels to IDs
        if config.label2id is not None and "label" in examples:
            result["label"] = [(config.label2id[l] if l != -1 else -1) for l in examples["label"]]
        return result

    datasets = datasets.map(preprocess_function, batched=True, load_from_cache_file=not data_args.overwrite_cache)
    # endregion

    with training_args.strategy.scope():
        # region Load pretrained model
        # Set seed before initializing model
        set_seed(training_args.seed)
        #
        # In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
        # download model & vocab.
        if checkpoint is None:
            model_path = model_args.model_name_or_path
        else:
            model_path = checkpoint
        model = TFAutoModelForSequenceClassification.from_pretrained(
            model_path,
            config=config,
            cache_dir=model_args.cache_dir,
            revision=model_args.model_revision,
            use_auth_token=True if model_args.use_auth_token else None,
        )
        # endregion

        # region Optimizer, loss and compilation
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=training_args.learning_rate,
            beta_1=training_args.adam_beta1,
            beta_2=training_args.adam_beta2,
            epsilon=training_args.adam_epsilon,
            clipnorm=training_args.max_grad_norm,
        )
        if is_regression:
            loss_fn = tf.keras.losses.MeanSquaredError()
            metrics = []
        else:
            loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            metrics = ["accuracy",f1_m,precision_m, recall_m]
        model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)
        # endregion

        # region Convert data to a tf.data.Dataset

        tf_data = dict()
        max_samples = {
            "train": data_args.max_train_samples,
            "validation": data_args.max_val_samples,
            "test": data_args.max_test_samples,
        }
        for key in ("train", "validation", "test"):
            if key not in datasets:
                tf_data[key] = None
                continue
            if key in ("train", "validation"):
                assert "label" in datasets[key].features, f"Missing labels from {key} data!"
            if key == "train":
                shuffle = True
                batch_size = training_args.per_device_train_batch_size
                drop_remainder = True  # Saves us worrying about scaling gradients for the last batch
            else:
                shuffle = False
                batch_size = training_args.per_device_eval_batch_size
                drop_remainder = False
            samples_limit = max_samples[key]
            dataset = datasets[key]
            if samples_limit is not None:
                dataset = dataset.select(range(samples_limit))
            if isinstance(training_args.strategy, tf.distribute.TPUStrategy) or data_args.pad_to_max_length:
                logger.info("Padding all batches to max length because argument was set or we're on TPU.")
                dataset_mode = "constant_batch"
            else:
                dataset_mode = "variable_batch"
            data = convert_dataset_for_tensorflow(
                dataset,
                non_label_column_names,
                batch_size=batch_size,
                dataset_mode=dataset_mode,
                drop_remainder=drop_remainder,
                shuffle=shuffle,
            )
            tf_data[key] = data
        # endregion

        # region Training and validation
        if tf_data["train"] is not None:
            callbacks = [SavePretrainedCallback(output_dir=training_args.output_dir)]
            model.fit(
                tf_data["train"],
                validation_data=tf_data["validation"],
                epochs=int(training_args.num_train_epochs),
                callbacks=callbacks,
            )
        elif tf_data["validation"] is not None:
            # If there's a validation dataset but no training set, just evaluate the metrics
            logger.info("Computing metrics on validation data...")
            if is_regression:
                loss = model.evaluate(tf_data["validation"])
                logger.info(f"Loss: {loss:.5f}")
            else:
                loss, accuracy, f1_score, precision, recall = model.evaluate(tf_data["validation"])
                logger.info(f"Loss: {loss:.5f}, Accuracy: {accuracy * 100:.4f}%, Precision: {precision}, Recall: {recall * 100:.4f}, F1: {f1_score}")
        
        # endregion

        # region Prediction
        if tf_data["test"] is not None:
            logger.info("Doing predictions on test dataset...")
            predictions = model.predict(tf_data["test"])["logits"]
            predicted_class = np.squeeze(predictions) if is_regression else np.argmax(predictions, axis=1)


            output_test_file = os.path.join(training_args.output_dir, "test_results.txt")
            with open(output_test_file, "w") as writer:
                writer.write("index\tprediction\n")
                for index, item in enumerate(predicted_class):
                    if is_regression:
                        writer.write(f"{index}\t{item:3.3f}\n")
                    else:
                        item = config.id2label[item]
                        writer.write(f"{index}\t{item}\n")
            logger.info(f"Wrote predictions to {output_test_file}!")
        # endregion

    # region Prediction losses
    # This section is outside the scope() because it's very quick to compute, but behaves badly inside it
    if "test" in datasets and "label" in datasets["test"].features:
        print("Computing prediction loss on test labels...")
        labels = datasets["test"]["label"]
        loss = float(loss_fn(labels, predictions).numpy())
        print(f"Test loss: {loss:.4f}")

    # endregion


if __name__ == "__main__":
    main()

Writing text_class.py


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("all_translated_data.csv", encoding = "utf-8")
data["claim"] = data["majority_label"].apply(lambda x: 1 if x == "Yes" else (0 if x == "No" else "No_Majority"))
# Just use the texts with a majority label
data = data[data.claim != "No_Majority"]
data = data.reset_index(drop=True)
data.head()

,Unnamed: 0,text,language,majority_label,source,annotator1,annotator2,annotator3,translated_texts,claim
0,0,ഇച്ഛാശക്തിയുള്ള ഒരു ഭരണകൂടത്തിന് അസാധ്യമായൊന്ന...,ml,Yes,social media,Yes,Yes,Yes,There is nothing impossible for a government t...,1
1,1,. *_SKY ViSiON_* / / ...,ml,Yes,social media,Yes,Yes,Yes,"== sync, corrected by elderman == @elder_man =...",1
2,2,+------+------+-----+------+-----+-----+------...,ml,Yes,social media,No,Yes,Yes,The-------------------------------------------...,1
3,4,ബിജെപി അല്ലെങ്കിൽ യുപിയിലെ സഖ്യം... ഇവരിൽ നിന്...,ml,Yes,social media,Yes,No,Yes,Bijepi or the party in Jupiter... any of them ...,1
4,5,. *_SKY ViSiON_* / / ...,ml,Yes,social media,Yes,Yes,Yes,*_SKY VISION_* / /_size_**_bulance_*_bulance_*...,1


In [ ]:
# Select all languages
data_en= data.loc[(data.language == "en")]
data_hi = data.loc[(data.language == "hi")]
data_bn = data.loc[(data.language == "bn")]
data_ta = data.loc[(data.language == "ta")]
data_ml = data.loc[(data.language == "ml")]

In [ ]:
# Create Train, Test, Validate for every language
data_en = data_en[["text", "claim"]]
data_hi = data_hi[["text", "claim"]]
data_ml = data_ml[["text", "claim"]]
data_ta = data_ta[["text", "claim"]]
data_bn = data_bn[["text", "claim"]]

data_en = data_en.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')
data_hi = data_hi.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')
data_ml = data_ml.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')
data_ta = data_ta.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')
data_bn = data_bn.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')

#set seed
np.random.seed(0)
train_en, validate_en, test_en = np.split(data_en.sample(frac=1), [int(.6*len(data_en)), int(.8*len(data_en))])
train_en.to_csv("train_en.csv")
validate_en.to_csv("validate_en.csv")
test_en.to_csv("test_en.csv")

#set seed
np.random.seed(0)
train_hi, validate_hi, test_hi = np.split(data_hi.sample(frac=1), [int(.6*len(data_hi)), int(.8*len(data_hi))])
train_hi.to_csv("train_hi.csv")
validate_hi.to_csv("validate_hi.csv")
test_hi.to_csv("test_hi.csv")

#set seed
np.random.seed(0)
train_ml, validate_ml, test_ml = np.split(data_ml.sample(frac=1), [int(.6*len(data_ml)), int(.8*len(data_ml))])
train_ml.to_csv("train_ml.csv")
validate_ml.to_csv("validate_ml.csv")
test_ml.to_csv("test_ml.csv")

#set seed
np.random.seed(0)
train_ta, validate_ta, test_ta = np.split(data_ta.sample(frac=1), [int(.6*len(data_ta)), int(.8*len(data_ta))])
train_ta.to_csv("train_ta.csv")
validate_ta.to_csv("validate_ta.csv")
test_ta.to_csv("test_ta.csv")

#set seed
np.random.seed(0)
train_bn, validate_bn, test_bn = np.split(data_bn.sample(frac=1), [int(.6*len(data_bn)), int(.8*len(data_bn))])
train_bn.to_csv("train_bn.csv")
validate_bn.to_csv("validate_bn.csv")
test_bn.to_csv("test_bn.csv")

In [ ]:
print(len(train_ml))
print(len(test_ml))
print(len(validate_ml))

582
194
194


In [ ]:
test_ml[:5]

,sentence1,label
621,തെളിരിക്കും !!! തെളിയിച്ചാല്‍ നിന്‍റെ നിലപാട...,0
461,കുഴഞ്ഞു വീഴുന്ന ജയ് ഹിന്ദ് ക്യാമറമാൻ.. താങ്ങി...,1
598,"നെല്ലാട്, കടക്കനാട്, തട്ടാമുകൾ, മഴുവന്നൂർ ഭാഗത...",0
139,[ ] [ ] സ്വാമി ശരണം SAVE...,1
749,. *_SKY VISION_* / / ...,1


In [ ]:
!python text_class.py \
--model_name_or_path distilbert-base-multilingual-cased \
--train_file train_en.csv \
--validation_file validate_en.csv \
--output_dir output_en/ \
--test_file test_en.csv

01/09/2022 11:27:18 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
test_res = pd.read_csv('output/test_results.txt', sep="\t")
test_versuch = pd.read_csv('test_versuch.csv')
test_res = test_res.prediction
test_versuch.label

0      0
1      1
2      1
3      0
4      0
      ..
180    1
181    0
182    1
183    1
184    1
Name: label, Length: 185, dtype: int64

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [ ]:
print("Classification report: \n", (classification_report(test_versuch.label, test_res)))
print("F1 micro averaging:",(f1_score(test_versuch.label, test_res, average='micro')))

Classification report: 
               precision    recall  f1-score   support

           0       0.95      0.74      0.83        84
           1       0.82      0.97      0.89       101

    accuracy                           0.86       185
   macro avg       0.89      0.85      0.86       185
weighted avg       0.88      0.86      0.86       185

F1 micro averaging: 0.8648648648648649


In [ ]:
!python text_class.py \
--model_name_or_path distilbert-base-multilingual-cased \
--train_file train_bn.csv \
--validation_file validate_bn.csv \
--output_dir output_bn/ \
--test_file test_bn.csv

01/09/2022 11:29:37 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
!python text_class.py \
--model_name_or_path distilbert-base-multilingual-cased \
--train_file train_hi.csv \
--validation_file validate_hi.csv \
--output_dir output_hi/ \
--test_file test_hi.csv

01/09/2022 11:31:25 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
!python text_class.py \
--model_name_or_path distilbert-base-multilingual-cased \
--train_file train_ml.csv \
--validation_file validate_ml.csv \
--output_dir output_ml/ \
--test_file test_ml.csv

01/09/2022 11:33:25 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
!python text_class.py \
--model_name_or_path distilbert-base-multilingual-cased \
--train_file train_ta.csv \
--validation_file validate_ta.csv \
--output_dir output_ta/ \
--test_file test_ta.csv

01/09/2022 11:35:45 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
test_en_res = pd.read_csv('output_en/test_results.txt', sep="\t")
test_en = pd.read_csv('test_en.csv')
test_en_res = test_en_res.prediction
test_en.label

print("EN: Classification report: \n", (classification_report(test_en.label, test_en_res)))
print("EN: F1 micro averaging:",(f1_score(test_en.label, test_en_res, average='micro')))

EN: Classification report: 
               precision    recall  f1-score   support

           0       1.00      0.68      0.81        84
           1       0.79      1.00      0.88       101

    accuracy                           0.85       185
   macro avg       0.89      0.84      0.85       185
weighted avg       0.88      0.85      0.85       185

EN: F1 micro averaging: 0.854054054054054


In [ ]:
test_hi_res = pd.read_csv('output_hi/test_results.txt', sep="\t")
test_hi = pd.read_csv('test_hi.csv')
test_hi_res = test_hi_res.prediction
test_hi.label

print("HI: Classification report: \n", (classification_report(test_hi.label, test_hi_res)))
print("HI: F1 micro averaging:",(f1_score(test_hi.label, test_hi_res, average='micro')))

HI: Classification report: 
               precision    recall  f1-score   support

           0       0.77      0.96      0.85        89
           1       0.94      0.71      0.81        86

    accuracy                           0.83       175
   macro avg       0.86      0.83      0.83       175
weighted avg       0.85      0.83      0.83       175

HI: F1 micro averaging: 0.8342857142857143


In [ ]:
test_ml_res = pd.read_csv('output_ml/test_results.txt', sep="\t")
test_ml = pd.read_csv('test_ml.csv')
test_ml_res = test_ml_res.prediction
test_ml.label

print("ML: Classification report: \n", (classification_report(test_ml.label, test_ml_res)))
print("ML: F1 micro averaging:",(f1_score(test_ml.label, test_ml_res, average='micro')))

ML: Classification report: 
               precision    recall  f1-score   support

           0       0.66      0.39      0.49        49
           1       0.82      0.93      0.87       145

    accuracy                           0.79       194
   macro avg       0.74      0.66      0.68       194
weighted avg       0.78      0.79      0.77       194

ML: F1 micro averaging: 0.7938144329896907


In [ ]:
test_ta_res = pd.read_csv('output_ta/test_results.txt', sep="\t")
test_ta = pd.read_csv('test_ta.csv')
test_ta_res = test_ta_res.prediction
test_ta.label

print("TA: Classification report: \n", (classification_report(test_ta.label, test_ta_res)))
print("TA: F1 micro averaging:",(f1_score(test_ta.label, test_ta_res, average='micro')))

TA: Classification report: 
               precision    recall  f1-score   support

           0       0.88      0.76      0.81        29
           1       0.83      0.92      0.87        37

    accuracy                           0.85        66
   macro avg       0.85      0.84      0.84        66
weighted avg       0.85      0.85      0.85        66

TA: F1 micro averaging: 0.8484848484848486


In [ ]:
test_bn_res = pd.read_csv('output_bn/test_results.txt', sep="\t")
test_bn = pd.read_csv('test_bn.csv')
test_bn_res = test_bn_res.prediction
test_bn.label

print("BN: Classification report: \n", (classification_report(test_bn.label, test_bn_res)))
print("BN: F1 micro averaging:",(f1_score(test_bn.label, test_bn_res, average='micro')))

BN: Classification report: 
               precision    recall  f1-score   support

           0       0.90      0.72      0.80       121
           1       0.61      0.84      0.71        64

    accuracy                           0.76       185
   macro avg       0.76      0.78      0.75       185
weighted avg       0.80      0.76      0.77       185

BN: F1 micro averaging: 0.7621621621621621
